In [15]:
import pandas as pd
from IPython.display import display, Markdown

from data_processing import get_correct_wrong_pred_df, get_category_gender_partition, get_false_preds

## Data

In [16]:
nlexique = pd.read_csv('../data/nlexique.csv')
nlexique = nlexique[['lexeme', 'sg', 'gen']].rename(columns={'sg': 'phon'}).dropna().reset_index(drop=True)
nlexique

,lexeme,phon,gen
0,à-côté,akOte,m
1,à-coup,aku,m
2,à-peu-près,apØpʁɛ,m
3,à-pic,apik,m
4,à-plat,apla,m
...,...,...,...
30999,zurichois,zyʁikwa,m
31000,zydeco,zidəko,f
31001,zygoma,zigOma,m
31002,zygote,zigɔt,m


In [17]:
echantinom = pd.read_csv('../data/Echantinom-full-20210902.csv')
echantinom = echantinom[~echantinom['lemma'].isin(nlexique[nlexique['gen'] == 'b']['lexeme'])] #[['lemma', 'phon', 'gen']]
echantinom

,lemma,gen,phon,freq_lex_books,freq_lex_subtitles,freq_frcow,last_process_broad,last_process_narrow,prefix,compound,...,autonomous_base,base_stem_phon,sfx_allomorph,der_stem_phon,edit_distance,pattern,pattern_tf,pattern_rel_tf,base_der_sim,offset_sim
0,berlingue,m,bɛʁ.lɛ̃g,0.34,0.00,34,nonconcat,apocope,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,corton,m,kɔʁ.tɔ̃,0.27,0.03,398,suffix,suffix,0,0,...,True,kuʁ,ɔ̃,kɔʁt,2,_u_~_ɔ_tɔ̃,1,0.015625,0.222162783145905,0.158108526129264
2,dabuche,f,da.byʃ,0.54,0.00,3,suffix,suffix,0,0,...,True,UNKNOWN,yʃ,dab,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
3,faf,m,faf,0.88,0.07,3422,nonconcat,apocope,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,gail,f,gaj,0.61,0.00,2471,simplex,native_simplex,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,palu,m,pa.ly,0.14,0.80,1431,nonconcat,apocope,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,talc,m,talk,1.49,1.40,2775,simplex,borrowing,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,sauvetage,m,sO.və.taʒ,3.72,8.32,60875,suffix,suffix,0,0,...,True,sOv,aʒ,sOvət,2,_~_ətaʒ,1,0.011494252873563,0.252674728631973,0.614848479997584
4998,plaid,m,plɛd,1.15,0.34,2889,simplex,borrowing,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
orth_preds_x10 = pd.read_csv('../results/echantinom_orth_preds_x10.csv')
orth_preds_x10.rename(columns={"Predicted Gender": "orth_pred", "True Gender": "true", "Form": 'lemma'}, inplace=True)
orth_preds_x10

,lemma,orth_pred,true,Class Probabilities,Set,Run
0,an,m,m,"[('n', {'f': 0.4085073173046112, 'm': 0.591492...",test,1
1,bi,m,m,"[('i', {'f': 0.16074861586093903, 'm': 0.83925...",test,1
2,as,m,m,"[('s', {'f': 0.2868475317955017, 'm': 0.713152...",test,1
3,té,m,m,"[('é', {'f': 0.3696044683456421, 'm': 0.630395...",test,1
4,pie,f,f,"[('e', {'f': 0.11107771843671799, 'm': 0.88892...",test,1
...,...,...,...,...,...,...
45315,approvisionnement,m,m,"[('t', {'f': 0.07809196412563324, 'm': 0.92190...",test,10
45316,sous-alimentation,f,f,"[('n', {'f': 0.38623788952827454, 'm': 0.61376...",test,10
45317,cul-de-basse-fosse,m,m,"[('e', {'f': 0.23680555820465088, 'm': 0.76319...",test,10
45318,marie-couche-toi-là,f,f,"[('à', {'f': 0.28149473667144775, 'm': 0.71850...",test,10


## Orthographic error analysis

In [20]:
print('\nCorrect and wrong orthographic predictions per gender:')
# print(get_correct_wrong_pred_df(orth_preds_x10, proportions=False).to_markdown(index=False))
distributions = get_correct_wrong_pred_df(orth_preds_x10, pred_col='orth_pred', proportions=False)
distributions_prop = get_correct_wrong_pred_df(orth_preds_x10, pred_col='orth_pred', proportions=True)
display(distributions, distributions_prop)


Correct and wrong orthographic predictions per gender:


,Run,f_true,m_true,f_false,m_false
0,1,1446,2674,199,213
1,2,1435,2665,208,224
2,3,1451,2663,210,208
3,4,1402,2721,152,257
4,5,1420,2692,181,239
5,6,1437,2660,213,222
6,7,1457,2645,228,202
7,8,1446,2663,210,213
8,9,1506,2585,288,153
9,10,1479,2643,230,180


,Run,f_true,m_true,f_false,m_false
0,1,0.879,0.926,0.121,0.074
1,2,0.873,0.922,0.127,0.078
2,3,0.874,0.928,0.126,0.072
3,4,0.902,0.914,0.098,0.086
4,5,0.887,0.918,0.113,0.082
5,6,0.871,0.923,0.129,0.077
6,7,0.865,0.929,0.135,0.071
7,8,0.873,0.926,0.127,0.074
8,9,0.839,0.944,0.161,0.056
9,10,0.865,0.936,0.135,0.064


### last_process_broad

In [22]:
category = 'last_process_broad' 
print(f"\nGender partition for {category} over all runs:")
all_runs = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred')
all_runs_proportions = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred', proportion=True)
display(all_runs, all_runs_proportions)


Gender partition for last_process_broad over all runs:


,f_true,m_true,f_false,m_false
last_process_broad,,,,
simplex,689,1254,98,114
suffix,608,1092,62,84
conversion,166,307,19,20
polylexical,78,161,8,10
nonconcat,39,57,3,9
prefix,28,53,6,7


,f_true,m_true,f_false,m_false
last_process_broad,,,,
simplex,0.875,0.917,0.125,0.083
suffix,0.907,0.929,0.093,0.071
conversion,0.897,0.939,0.103,0.061
polylexical,0.907,0.942,0.093,0.058
nonconcat,0.929,0.864,0.071,0.136
prefix,0.824,0.883,0.176,0.117


In [24]:
run = 1

print(f"\n [Run {run}] Gender partition for {category}:")
run1 = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred', run=run)
run1_prop = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred', run=run, proportion=True)
display(run1, run1_prop)


 [Run 1] Gender partition for last_process_broad:


,f_true,m_true,f_false,m_false
last_process_broad,,,,
simplex,635,1117,92,87
suffix,565,963,53,70
conversion,159,285,16,15
polylexical,74,144,8,8
nonconcat,34,50,3,6
prefix,25,51,6,5


,f_true,m_true,f_false,m_false
last_process_broad,,,,
simplex,0.873,0.928,0.127,0.072
suffix,0.914,0.932,0.086,0.068
conversion,0.909,0.950,0.091,0.050
polylexical,0.902,0.947,0.098,0.053
nonconcat,0.919,0.893,0.081,0.107
prefix,0.806,0.911,0.194,0.089


In [80]:
run_data = orth_preds_x10[orth_preds_x10['Run'] == 3]
crosstab = pd.crosstab(echantinom['last_process_broad'], run_data['true'])
# TODO: why nothing from run 3 onwards?
crosstab = crosstab.loc[crosstab.sum(axis=1).sort_values(ascending=False).index]
crosstab

true
last_process_broad


#### False f

In [26]:
run = 1
pred_gender = 'f'
true_gender = 'm'
category = 'last_process_broad'

f_false_rows = orth_preds_x10[(orth_preds_x10['Run'] == run) & (orth_preds_x10["orth_pred"] == pred_gender) & (orth_preds_x10['true'] == true_gender)]

# Merge the f_false_rows with the echantinom DataFrame to get the 'last_process_broad' column
f_false_rows = f_false_rows.merge(echantinom[['lemma', category]], how='left', left_on='lemma', right_on='lemma')

f_false_rows.groupby(category)['lemma'].count().sort_values(ascending=False)
# f_false_rows

last_process_broad
simplex        138
polylexical     19
suffix          15
conversion      13
nonconcat        8
prefix           6
Name: lemma, dtype: int64

In [29]:
subcategory = 'simplex'

print(f'\nExploring {pred_gender}_false:')
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
# TODO: 138 rows where we're supposed to get 92 f_false
false_rows['lemma'].unique()


Exploring f_false:
[Run 1] f_false simplex count: 138


array(['site', 'cube', 'mime', 'vice', 'vide', 'dine', 'vote', 'pèze',
       'mile', 'cola', 'culte', 'drone', 'amble', 'limbe', 'nonce',
       'monde', 'fifre', 'clone', 'ongle', 'nimbe', 'prote', 'torse',
       'blase', 'palpe', 'galbe', 'renne', 'orgue', 'morse', 'birbe',
       'tison', 'pence', 'grime', 'baile', 'gypse', 'angle', 'rifle',
       'saule', 'arôme', 'kyrie', 'sauna', 'drive', 'tulle', 'sosie',
       'gamma', 'pagne', 'agrume', 'prêche', 'caïque', 'vergne', 'litige',
       'madère', 'porche', 'baffle', 'pulque', 'curare', 'flegme',
       'cigare', 'lierre', 'calque', 'latino', 'buffle', 'cirque',
       'casque', 'poison', 'stupre', 'couple', 'junkie', 'rallye',
       'druide', 'cierge', 'congre', 'zouave', 'lambda', 'chèche',
       'drille', 'bidule', 'causse', 'comble', 'jacques', 'vampire',
       'tumulte', 'bidasse', 'bacille', 'guinche', 'concile', 'frisbee',
       'silence', 'gruyère', 'emblème', 'fourgue', 'gymnase', 'marsala',
       'braille', 'murm

In [30]:
subcategory = 'polylexical'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false polylexical count: 19


array(['adverbe', 'fanzine', 'confrère', 'unetelle', 'discobole',
       'bidonville', 'bain-marie', 'passe-droit', 'saint-pierre',
       'portefeuille', 'chasse-neige', 'faire-valoir', 'claque-merde',
       'mille-feuille', 'chèvrefeuille', 'soutien-gorge', 'croquemitaine',
       'homme-grenouille', 'contre-la-montre'], dtype=object)

In [31]:
subcategory = 'suffix'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false suffix count: 15


array(['ovule', 'butane', 'nivôse', 'pendule', 'sulfure', 'cyanure',
       'lactose', 'ventôse', 'lignite', 'globule', 'caniche', 'fascicule',
       'demandeur', 'capitaine', 'nourrisson'], dtype=object)

In [32]:
subcategory = 'conversion'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false conversion count: 13


array(['onze', 'sauté', 'double', 'balaise', 'parjure', 'atlante',
       'immeuble', 'uniforme', 'acquitté', 'burlesque', 'maxillaire',
       'plantigrade', 'barbiturique'], dtype=object)

#### False m

In [33]:
run = 1
pred_gender = 'm'
true_gender = 'f'
category = 'last_process_broad'

print(f'\nExploring {pred_gender}_false:')
false_rows = orth_preds_x10[(orth_preds_x10['Run'] == run) & (orth_preds_x10["orth_pred"] == pred_gender) & (orth_preds_x10['true'] == true_gender)]
false_rows = false_rows.merge(echantinom[['lemma', category]], how='left', left_on='lemma', right_on='lemma')
false_rows.groupby(category)['lemma'].count().sort_values(ascending=False)
# false_rows


Exploring m_false:


last_process_broad
simplex        147
suffix          27
nonconcat       14
polylexical     12
prefix           8
conversion       5
Name: lemma, dtype: int64

In [34]:
subcategory = 'simplex'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false simplex count: 147


array(['foi', 'ire', 'nef', 'ive', 'glu', 'bru', 'obi', 'mer', 'clé',
       'eau', 'cage', 'loge', 'robe', 'cour', 'soif', 'miss', 'care',
       'tong', 'girl', 'part', 'gail', 'nage', 'rage', 'diva', 'toge',
       'puce', 'zone', 'kipa', 'sape', 'hâte', 'city', 'soul', 'acre',
       'cave', 'noix', 'gent', 'tribu', 'piste', 'câpre', 'douma',
       'serre', 'façon', 'spore', 'hydre', 'pogne', 'bugle', 'fleur',
       'corne', 'vodka', 'terre', 'vertu', 'bible', 'ancre', 'flore',
       'brume', 'spire', 'bribe', 'alène', 'nacre', 'taule', 'jauge',
       'grâce', 'plume', 'horde', 'kacha', 'smala', 'squaw', 'trame',
       'savate', 'carène', 'satire', 'trique', 'huître', 'baraka',
       'guenon', 'frange', 'truite', 'tumeur', 'sangle', 'capote',
       'armada', 'razzia', 'pagode', 'strate', 'alcôve', 'galène',
       'crypte', 'igname', 'cadène', 'poudre', 'brebis', 'gabare',
       'bastos', 'lymphe', 'pin-up', 'cloque', 'rumeur', 'vahiné',
       'fenêtre', 'syllabe', 'pirogu

In [35]:
subcategory = 'suffix'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false suffix count: 27


array(['catin', 'sueur', 'pudeur', 'fureur', 'piqûre', 'levure',
       'vigueur', 'parenté', 'candeur', 'minceur', 'ferveur', 'boisson',
       'louange', 'chanson', 'chaleur', 'rondeur', 'blondeur', 'noirceur',
       'passoire', 'rousseur', 'grandeur', 'splendeur', 'blancheur',
       'bronchite', 'corpuscule', 'bassinoire', 'rôtissoire'],
      dtype=object)

In [36]:
subcategory = 'nonconcat'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false nonconcat count: 14


array(['mob', 'info', 'cata', 'keuf', 'mayo', 'diapo', 'conso', 'impro',
       'philo', 'nympho', 'porcif', 'nounou', 'dondon', 'thalasso'],
      dtype=object)

In [37]:
subcategory = 'polylexical'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false polylexical count: 12


array(['madame', 'silicone', 'parabole', 'bande-son', 'pause-café',
       'notre-dame', "presqu'île", 'claire-voie', 'flanc-garde',
       'grand-route', 'grand-voile', 'tête-de-mort'], dtype=object)

### last_process_narrow

In [38]:
category = 'last_process_narrow' 
print(f"\nGender partition for {category} over all runs:")
all_runs = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred')
all_runs_proportions = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred', proportion=True)
display(all_runs, all_runs_proportions)


Gender partition for last_process_narrow over all runs:


,f_true,m_true,f_false,m_false
last_process_narrow,,,,
suffix,608,1092,62.0,84.0
native_simplex,504,935,78.0,89.0
borrowing,144,265,18.0,22.0
conversion-A,139,231,14.0,18.0
native_compound,46,101,6.0,7.0
prefix,28,53,6.0,7.0
antonomasia,38,50,2.0,2.0
neoclassical_compound,14,28,1.0,3.0
agglomerate,17,25,1.0,0.0


,f_true,m_true,f_false,m_false
last_process_narrow,,,,
suffix,0.907,0.929,0.093,0.071
native_simplex,0.866,0.913,0.134,0.087
borrowing,0.889,0.923,0.111,0.077
conversion-A,0.908,0.928,0.092,0.072
native_compound,0.885,0.935,0.115,0.065
prefix,0.824,0.883,0.176,0.117
antonomasia,0.950,0.962,0.050,0.038
neoclassical_compound,0.933,0.903,0.067,0.097
agglomerate,0.944,1.000,0.056,0.000


#### False f

In [39]:
run = 1
pred_gender = 'f'
true_gender = 'm'
category = 'last_process_narrow'

f_false_rows = orth_preds_x10[(orth_preds_x10['Run'] == run) & (orth_preds_x10["orth_pred"] == pred_gender) & (orth_preds_x10['true'] == true_gender)]
f_false_rows = f_false_rows.merge(echantinom[['lemma', category]], how='left', left_on='lemma', right_on='lemma')

f_false_rows.groupby(category)['lemma'].count().sort_values(ascending=False)
# f_false_rows

last_process_narrow
native_simplex           103
borrowing                 22
suffix                    15
antonomasia               13
native_compound           13
conversion-A              12
prefix                     6
apocope                    3
apocope_with_appendix      3
neoclassical_compound      3
agglomerate                2
blend                      1
conversion-NUM             1
procope                    1
reduplication              1
Name: lemma, dtype: int64

In [40]:
subcategory = 'native_simplex'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false native_simplex count: 103


array(['site', 'cube', 'mime', 'vice', 'vide', 'dine', 'vote', 'pèze',
       'mile', 'culte', 'drone', 'amble', 'limbe', 'nonce', 'monde',
       'fifre', 'clone', 'ongle', 'nimbe', 'prote', 'torse', 'blase',
       'palpe', 'galbe', 'renne', 'orgue', 'morse', 'birbe', 'tison',
       'grime', 'baile', 'gypse', 'angle', 'saule', 'arôme', 'tulle',
       'pagne', 'agrume', 'prêche', 'vergne', 'litige', 'porche',
       'curare', 'flegme', 'cigare', 'lierre', 'calque', 'buffle',
       'cirque', 'casque', 'poison', 'stupre', 'couple', 'druide',
       'cierge', 'congre', 'zouave', 'chèche', 'drille', 'bidule',
       'causse', 'comble', 'vampire', 'tumulte', 'bidasse', 'bacille',
       'guinche', 'concile', 'silence', 'emblème', 'fourgue', 'gymnase',
       'murmure', 'salaire', 'symbole', 'falbala', 'flingue', 'bastion',
       'vacarme', 'chevesne', 'cantique', 'grimoire', 'quarante',
       'genièvre', 'conclave', 'margrave', 'triomphe', 'scarabée',
       'carrosse', 'brahmane', 'c

In [41]:
subcategory = 'borrowing'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false borrowing count: 22


array(['cola', 'pence', 'rifle', 'kyrie', 'sauna', 'drive', 'gamma',
       'caïque', 'baffle', 'pulque', 'latino', 'junkie', 'rallye',
       'lambda', 'frisbee', 'bouddha', 'bazooka', 'folklore', 'barbecue',
       'dies irae', 'cheese-cake', 'strip-tease'], dtype=object)

In [42]:
subcategory = 'suffix'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false suffix count: 15


array(['ovule', 'butane', 'nivôse', 'pendule', 'sulfure', 'cyanure',
       'lactose', 'ventôse', 'lignite', 'globule', 'caniche', 'fascicule',
       'demandeur', 'capitaine', 'nourrisson'], dtype=object)

In [43]:
subcategory = 'antonomasia'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false antonomasia count: 13


array(['sosie', 'madère', 'jacques', 'gruyère', 'marsala', 'braille',
       'mercure', 'narcisse', 'matamore', 'macintosh', 'champagne',
       'bourgogne', 'macfarlane'], dtype=object)

In [44]:
subcategory = 'native_compound'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false native_compound count: 13


array(['bidonville', 'bain-marie', 'passe-droit', 'saint-pierre',
       'portefeuille', 'chasse-neige', 'faire-valoir', 'claque-merde',
       'mille-feuille', 'chèvrefeuille', 'soutien-gorge', 'croquemitaine',
       'homme-grenouille'], dtype=object)

In [45]:
subcategory = 'conversion-A'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false conversion-A count: 12


array(['sauté', 'double', 'balaise', 'parjure', 'atlante', 'immeuble',
       'uniforme', 'acquitté', 'burlesque', 'maxillaire', 'plantigrade',
       'barbiturique'], dtype=object)

In [46]:
subcategory = 'prefix'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false prefix count: 6


array(['vicomte', 'tricorne', 'demi-frère', 'entrecuisse', 'arrière-goût',
       'contrepoison'], dtype=object)

#### False m

In [47]:
run = 1
pred_gender = 'm'
true_gender = 'f'
category = 'last_process_narrow'

f_false_rows = orth_preds_x10[(orth_preds_x10['Run'] == run) & (orth_preds_x10["orth_pred"] == pred_gender) & (orth_preds_x10['true'] == true_gender)]

# Merge the f_false_rows with the echantinom DataFrame to get the 'last_process_broad' column
f_false_rows = f_false_rows.merge(echantinom[['lemma', category]], how='left', left_on='lemma', right_on='lemma')

f_false_rows.groupby(category)['lemma'].count().sort_values(ascending=False)
# f_false_rows

last_process_narrow
native_simplex           118
borrowing                 27
suffix                    27
apocope                   10
prefix                     8
native_compound            7
conversion-A               5
agglomerate                3
antonomasia                2
neoclassical_compound      2
reduplication              2
apocope_with_appendix      1
verlan                     1
Name: lemma, dtype: int64

In [48]:
subcategory = 'native_simplex'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false native_simplex count: 118


array(['foi', 'ire', 'nef', 'ive', 'glu', 'bru', 'mer', 'clé', 'eau',
       'cage', 'loge', 'robe', 'cour', 'soif', 'care', 'part', 'gail',
       'nage', 'rage', 'toge', 'puce', 'zone', 'sape', 'hâte', 'acre',
       'cave', 'noix', 'gent', 'tribu', 'piste', 'câpre', 'serre',
       'façon', 'spore', 'hydre', 'pogne', 'bugle', 'fleur', 'corne',
       'terre', 'vertu', 'bible', 'ancre', 'flore', 'brume', 'spire',
       'bribe', 'alène', 'nacre', 'taule', 'jauge', 'grâce', 'plume',
       'horde', 'trame', 'savate', 'carène', 'satire', 'trique', 'huître',
       'guenon', 'frange', 'truite', 'tumeur', 'sangle', 'capote',
       'pagode', 'strate', 'alcôve', 'galène', 'crypte', 'igname',
       'cadène', 'poudre', 'brebis', 'gabare', 'lymphe', 'cloque',
       'rumeur', 'fenêtre', 'syllabe', 'pirogue', 'arnaque', 'cuiller',
       'septime', 'riposte', 'tartane', 'bagarre', 'victime', 'imposte',
       'horloge', 'couleur', 'attaque', 'mandore', 'fanfare', 'chicane',
       'enclume',

In [49]:
subcategory = 'borrowing'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false borrowing count: 27


array(['obi', 'miss', 'tong', 'girl', 'diva', 'kipa', 'city', 'soul',
       'douma', 'vodka', 'kacha', 'smala', 'squaw', 'baraka', 'armada',
       'razzia', 'pin-up', 'vahiné', 'canasta', 'alhambra', 'start-up',
       'fantasia', 'maharani', 'williams', 'superstar', 'trattoria',
       'garden-party'], dtype=object)

In [50]:
subcategory = 'suffix'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false suffix count: 27


array(['catin', 'sueur', 'pudeur', 'fureur', 'piqûre', 'levure',
       'vigueur', 'parenté', 'candeur', 'minceur', 'ferveur', 'boisson',
       'louange', 'chanson', 'chaleur', 'rondeur', 'blondeur', 'noirceur',
       'passoire', 'rousseur', 'grandeur', 'splendeur', 'blancheur',
       'bronchite', 'corpuscule', 'bassinoire', 'rôtissoire'],
      dtype=object)

In [51]:
subcategory = 'apocope'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false apocope count: 10


array(['mob', 'info', 'cata', 'mayo', 'diapo', 'conso', 'impro', 'philo',
       'nympho', 'thalasso'], dtype=object)

In [52]:
subcategory = 'prefix'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false prefix count: 8


array(['malfaçon', 'ex-femme', 'mi-pente', 'mini-jupe', 'mi-carême',
       'mi-juillet', 'contrefaçon', 'avant-première'], dtype=object)

### Compound

In [53]:
category = 'compound' 
print(f"\nGender partition for {category} over all runs:")
all_runs = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred')
all_runs_proportions = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred', proportion=True)
display(all_runs, all_runs_proportions)


Gender partition for compound over all runs:


,f_true,m_true,f_false,m_false
compound,,,,
0,1540,2774,188.0,233.0
neoclassical,20,42,1.0,4.0
VERB-NOUN,15,33,4.0,3.0
NOUN-NOUN,15,27,1.0,1.0
ADJ-NOUN,11,24,1.0,2.0
NOUN-ADJ,2,12,0.0,1.0
ADJ-ADJ,1,4,0.0,0.0
VERB-ADV,2,2,0.0,0.0
ADV-ADJ,1,1,1.0,0.0


,f_true,m_true,f_false,m_false
compound,,,,
0,0.891,0.923,0.109,0.077
neoclassical,0.952,0.913,0.048,0.087
VERB-NOUN,0.789,0.917,0.211,0.083
NOUN-NOUN,0.938,0.964,0.062,0.036
ADJ-NOUN,0.917,0.923,0.083,0.077
NOUN-ADJ,1.000,0.923,0.000,0.077
ADJ-ADJ,1.000,1.000,0.000,0.000
VERB-ADV,1.000,1.000,0.000,0.000
ADV-ADJ,0.500,1.000,0.500,0.000


#### False f

In [54]:
run = 1
pred_gender = 'f'
true_gender = 'm'
category = 'compound'

f_false_rows = orth_preds_x10[(orth_preds_x10['Run'] == run) & (orth_preds_x10["orth_pred"] == pred_gender) & (orth_preds_x10['true'] == true_gender)]
f_false_rows = f_false_rows.merge(echantinom[['lemma', category]], how='left', left_on='lemma', right_on='lemma')

f_false_rows.groupby(category)['lemma'].count().sort_values(ascending=False)
# f_false_rows

compound
0               182
VERB-NOUN         6
NOUN-NOUN         5
neoclassical      4
ADJ-NOUN          1
VERB-VERB         1
Name: lemma, dtype: int64

In [55]:
subcategory = 'VERB-NOUN'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false VERB-NOUN count: 6


array(['passe-droit', 'portefeuille', 'chasse-neige', 'claque-merde',
       'soutien-gorge', 'croquemitaine'], dtype=object)

In [56]:
subcategory = 'NOUN-NOUN'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false NOUN-NOUN count: 5


array(['bidonville', 'bain-marie', 'saint-pierre', 'chèvrefeuille',
       'homme-grenouille'], dtype=object)

In [57]:
subcategory = 'neoclassical'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false neoclassical count: 4


array(['adverbe', 'confrère', 'discobole', 'plantigrade'], dtype=object)

#### False m

In [58]:
run = 1
pred_gender = 'm'
true_gender = 'f'
category = 'compound'

f_false_rows = orth_preds_x10[(orth_preds_x10['Run'] == run) & (orth_preds_x10["orth_pred"] == pred_gender) & (orth_preds_x10['true'] == true_gender)]
f_false_rows = f_false_rows.merge(echantinom[['lemma', category]], how='left', left_on='lemma', right_on='lemma')

f_false_rows.groupby(category)['lemma'].count().sort_values(ascending=False)
# f_false_rows

compound
0               201
neoclassical      5
ADJ-NOUN          3
NOUN-NOUN         3
ADV-NOUN          1
Name: lemma, dtype: int64

In [59]:
subcategory = 'neoclassical'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false neoclassical count: 5


array(['philo', 'nympho', 'thalasso', 'silicone', 'parabole'],
      dtype=object)

In [60]:
subcategory = 'ADJ-NOUN'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false ADJ-NOUN count: 3


array(['claire-voie', 'grand-route', 'grand-voile'], dtype=object)

In [61]:
subcategory = 'NOUN-NOUN'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false NOUN-NOUN count: 3


array(['bande-son', 'pause-café', 'flanc-garde'], dtype=object)

### Suffix broad

In [62]:
category = 'suffix_broad' 
print(f"\nGender partition for {category} over all runs:")
all_runs = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred')
all_runs_proportions = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred', proportion=True)
display(all_runs, all_runs_proportions)


Gender partition for suffix_broad over all runs:


,f_true,m_true,f_false,m_false
suffix_broad,,,,
0,930,1725,124.0,148.0
eurM1,62,126,9.0,9.0
ment,73,111,8.0,10.0
ion,58,104,3.0,11.0
ier,52,82,3.0,7.0
...,...,...,...,...
en,1,0,0.0,0.0
one,0,1,0.0,0.0
aque,1,0,0.0,0.0


,f_true,m_true,f_false,m_false
suffix_broad,,,,
0,0.882,0.921,0.118,0.079
eurM1,0.873,0.933,0.127,0.067
ment,0.901,0.917,0.099,0.083
ion,0.951,0.904,0.049,0.096
ier,0.945,0.921,0.055,0.079
...,...,...,...,...
en,1.000,NaN,0.000,NaN
one,NaN,1.000,NaN,0.000
aque,1.000,NaN,0.000,NaN


#### False f

In [63]:
run = 1
pred_gender = 'f'
true_gender = 'm'
category = 'suffix_broad'

f_false_rows = orth_preds_x10[(orth_preds_x10['Run'] == run) & (orth_preds_x10["orth_pred"] == pred_gender) & (orth_preds_x10['true'] == true_gender)]
f_false_rows = f_false_rows.merge(echantinom[['lemma', category]], how='left', left_on='lemma', right_on='lemma')
f_false_rows.groupby(category)['lemma'].count().sort_values(ascending=False)


suffix_broad
0        181
ule        3
Vche       2
ain        2
ureM       2
ôse        2
aneM       1
cule       1
eurM1      1
ique       1
ite        1
on         1
ose        1
Name: lemma, dtype: int64

In [64]:
subcategory = 'ule'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false ule count: 3


array(['ovule', 'pendule', 'globule'], dtype=object)

In [65]:
subcategory = 'Vche'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false Vche count: 2


array(['matuche', 'caniche'], dtype=object)

In [66]:
subcategory = 'ureM'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false ureM count: 2


array(['sulfure', 'cyanure'], dtype=object)

#### False m

In [67]:
run = 1
pred_gender = 'm'
true_gender = 'f'
category = 'suffix_broad'

print(f'\nExploring {pred_gender}_false:')
false_rows = orth_preds_x10[(orth_preds_x10['Run'] == run) & (orth_preds_x10["orth_pred"] == pred_gender) & (orth_preds_x10['true'] == true_gender)]
false_rows = false_rows.merge(echantinom[['lemma', category]], how='left', left_on='lemma', right_on='lemma')
false_rows.groupby(category)['lemma'].count().sort_values(ascending=False)
# false_rows


Exploring m_false:


suffix_broad
0       182
eurF     15
onF       4
oir       3
ureF      2
aire      1
ange      1
cule      1
if        1
in        1
ite       1
ité       1
Name: lemma, dtype: int64

In [68]:
subcategory = 'eurF'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false eurF count: 15


array(['sueur', 'pudeur', 'fureur', 'vigueur', 'candeur', 'minceur',
       'ferveur', 'chaleur', 'rondeur', 'blondeur', 'noirceur',
       'rousseur', 'grandeur', 'splendeur', 'blancheur'], dtype=object)

In [69]:
subcategory = 'onF'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false onF count: 4


array(['boisson', 'chanson', 'malfaçon', 'contrefaçon'], dtype=object)

In [70]:
subcategory = 'oir'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false oir count: 3


array(['passoire', 'bassinoire', 'rôtissoire'], dtype=object)

### Conversion

In [71]:
category = 'conversion' 
print(f"\nGender partition for {category} over all runs:")
all_runs = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred')
all_runs_proportions = get_category_gender_partition(category, echantinom, orth_preds_x10, pred_col='orth_pred', proportion=True)
display(all_runs, all_runs_proportions)


Gender partition for conversion over all runs:


,f_true,m_true,f_false,m_false
conversion,,,,
0,1151,2061,140.0,177.0
V0,222,446,32.0,32.0
A,219,363,21.0,33.0
V12,6,28,2.0,1.0
V13,2,11,0.0,0.0
VINF,2,6,0.0,0.0
ADV,2,3,0.0,0.0
N,2,2,0.0,1.0
V,2,2,0.0,0.0


,f_true,m_true,f_false,m_false
conversion,,,,
0,0.892,0.921,0.108,0.079
V0,0.874,0.933,0.126,0.067
A,0.912,0.917,0.088,0.083
V12,0.750,0.966,0.250,0.034
V13,1.000,1.000,0.000,0.000
VINF,1.000,1.000,0.000,0.000
ADV,1.000,1.000,0.000,0.000
N,1.000,0.667,0.000,0.333
V,1.000,1.000,0.000,0.000


#### False f

In [72]:
run = 1
pred_gender = 'f'
true_gender = 'm'
category = 'conversion'

f_false_rows = orth_preds_x10[(orth_preds_x10['Run'] == run) & (orth_preds_x10["orth_pred"] == pred_gender) & (orth_preds_x10['true'] == true_gender)]
f_false_rows = f_false_rows.merge(echantinom[['lemma', category]], how='left', left_on='lemma', right_on='lemma')

f_false_rows.groupby(category)['lemma'].count().sort_values(ascending=False)

conversion
0      152
V0      34
A       12
NUM      1
Name: lemma, dtype: int64

In [73]:
subcategory = 'V0'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false V0 count: 34


array(['cube', 'mime', 'vice', 'vide', 'vote', 'pèze', 'amble', 'fifre',
       'clone', 'nimbe', 'palpe', 'galbe', 'tison', 'grime', 'angle',
       'prêche', 'calque', 'casque', 'couple', 'comble', 'sulfure',
       'tumulte', 'guinche', 'silence', 'cyanure', 'fourgue', 'murmure',
       'salaire', 'flingue', 'vacarme', 'triomphe', 'carrosse',
       'matamore', 'bastringue'], dtype=object)

In [74]:
subcategory = 'A'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] f_false A count: 12


array(['sauté', 'double', 'balaise', 'parjure', 'atlante', 'immeuble',
       'uniforme', 'acquitté', 'burlesque', 'maxillaire', 'plantigrade',
       'barbiturique'], dtype=object)

#### False m

In [75]:
run = 1
pred_gender = 'm'
true_gender = 'f'
category = 'conversion'

f_false_rows = orth_preds_x10[(orth_preds_x10['Run'] == run) & (orth_preds_x10["orth_pred"] == pred_gender) & (orth_preds_x10['true'] == true_gender)]
f_false_rows = f_false_rows.merge(echantinom[['lemma', category]], how='left', left_on='lemma', right_on='lemma')

f_false_rows.groupby(category)['lemma'].count().sort_values(ascending=False)

conversion
0     158
V0     48
A       7
Name: lemma, dtype: int64

In [76]:
subcategory = 'V0'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false V0 count: 48


array(['glu', 'loge', 'robe', 'soif', 'nage', 'rage', 'zone', 'sape',
       'hâte', 'cave', 'piste', 'serre', 'façon', 'pogne', 'fleur',
       'corne', 'terre', 'ancre', 'brume', 'nacre', 'jauge', 'grâce',
       'plume', 'trame', 'carène', 'trique', 'frange', 'sangle', 'capote',
       'poudre', 'cloque', 'levure', 'fenêtre', 'syllabe', 'arnaque',
       'riposte', 'louange', 'bagarre', 'victime', 'couleur', 'attaque',
       'fanfare', 'chicane', 'applique', 'histoire', 'vertèbre',
       'silicone', 'harangue'], dtype=object)

In [77]:
subcategory = 'A'
false_rows= get_false_preds(run, echantinom, 'orth_pred', pred_gender, true_gender, orth_preds_x10, category, subcategory)
print(f"[Run {run}] {pred_gender}_false {subcategory} count: {len(false_rows['lemma'].unique())}")
false_rows['lemma'].unique()

[Run 1] m_false A count: 7


array(['nympho', 'dextre', 'mormone', 'secrète', 'bayadère', 'verticale',
       'perpendiculaire'], dtype=object)